Pedro Ivo Vasconcelos - TP

## **Capítulo 5**

1) Critério de Jury: $$G(z) = \frac{0.6177}{z - 0.7770}$$

<img alt="Lugar das Raízes" src="anexos\c5e1 Root Locus.png" title="Lugar das Raízes">


2) Nyquist:

<img alt="Nyquist" src="anexos\c5e2 Nyquist.png" title="Nyquist">


3) Bode: O valor não possui a margem de fase desejada, ao saturar
 em -180º a partir da amostragem o sistema se mostra incapaz de
 ultrapassar a zona instável.

<img alt="Diagrama de Bode" src="anexos\c5e3 Diagrama de Bode.png" title="Diagrama de Bode">

4) Nichols:

<img alt="Nichols" src="anexos\c5e4 Nichols.png" title="Nichols">

5) Lugar das Raízes: O valor se mostra coerente, pois a linha está sobre um número ímpar do somatório de pólos e zeros.